In [27]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

# Construct the relative file path
file_path = os.path.join('..', '..', 'Output', 'Simulation', 'quadcopterFeasability.csv')
data = pd.read_csv(file_path)
file_path = os.path.join('..', '..', 'Output', 'Simulation', 'simulation-WMU501-varEffort-comparsiondf.csv')
comparison_df = pd.read_csv(file_path)

# Ensure the columns align by merging the two DataFrames on the matching columns
merged_df = data.merge(
    comparison_df[['Number_of_Plots', 'Mean_Cover_Percentage']],
    left_on='subplots',
    right_on='Number_of_Plots',
    how='left'
)

# Replace the 'area_p' column in the original DataFrame with 'Mean_Cover_Percentage'
data['area_p'] = merged_df['Mean_Cover_Percentage']



# Extract columns
area = data['area_p']
density = data['trueDensity']
detections = data['mean_n']

# Define colors based on the number of detections
colors = []
for detection in detections:
    if detection < 60:
        colors.append('red')
    elif 60 <= detection <= 80:
        colors.append('yellow')
    else:
        colors.append('green')

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=area,
    y=density,
    z=detections,
    mode='markers',
    marker=dict(
        size=5,
        color=colors,  # set color to an array/list of desired values
        opacity=0.8
    )
)])

# Labels and tick marks
fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='Relative area covered',
            tickmode='linear',
            dtick=5.0  # Set ticks in 0.2 increments
        ),
        yaxis=dict(
            title='Density (animals/1000km^2)'
        ),
        zaxis_title='Number of Detections'
    )
)

# Define the x-axis regions
x_orange = np.linspace(10, 15, 50)  # Orange region between 8 and 16
x_red = np.linspace(0, 10, 50)      # Red region below 8

# Define y and z ranges for the shaded regions
y_range = np.linspace(density.min(), density.max(), 50)
z_range = np.linspace(detections.min(), detections.max(), 50)

# Create mesh grids for the orange region
x_orange_grid, y_orange_grid = np.meshgrid(x_orange, y_range)
z_orange_grid = np.full_like(x_orange_grid, detections.min())  # Flat plane at the bottom

# Create mesh grids for the red region
x_red_grid, y_red_grid = np.meshgrid(x_red, y_range)
z_red_grid = np.full_like(x_red_grid, detections.min())  # Flat plane at the bottom

# Add the orange region as a surface
fig.add_trace(go.Surface(
    x=x_orange_grid,
    y=y_orange_grid,
    z=z_orange_grid,
    colorscale=[[0, 'orange'], [1, 'orange']],
    opacity=0.3,
    showscale=False
))

# Add the red region as a surface
fig.add_trace(go.Surface(
    x=x_red_grid,
    y=y_red_grid,
    z=z_red_grid,
    colorscale=[[0, 'red'], [1, 'red']],
    opacity=0.3,
    showscale=False
))

# Show plot
fig.show()

# Save the plot as an HTML file
output_html_path = os.path.join('..', '..', 'Output', 'Plots', 'quadcopterFeasability_plot.html')
pio.write_html(fig, output_html_path)

In [21]:
data

,subplots,mean_n,area,trueDensity,d_number,area_p
0,15,12,190,75.159,0.25,8.55
1,15,22,190,150.319,0.50,8.55
2,15,33,190,225.478,0.75,8.55
3,15,39,190,300.638,1.00,8.55
4,15,79,190,601.275,2.00,8.55
5,15,158,190,1202.551,4.00,8.55
6,15,316,190,2405.101,8.00,8.55
7,31,19,392,58.051,0.25,17.68
8,31,38,392,116.102,0.50,17.68
9,31,55,392,174.154,0.75,17.68


In [19]:
comparison_df

,Simulation,Design,Mean_Sampler_Count,Mean_Cover_Area,Mean_Cover_Percentage,Mean_Line_Length,Mean_Trackline_Length,Mean_Cyclic_Trackline_Length,Mean_On_Effort,Mean_Off_Effort,Mean_Return_to_Home,Mean_Off_Effort_Return,On_Effort_Percentage,Off_Effort_Percentage,Return_to_Home_Percentage,Off_Effort_Return_Percentage,Number_of_Plots
0,10L,systematic,10.0,1.702607e+08,7.975704,329112.05,448863.69,509692.11,329112.05,119751.64,60828.42,180580.06,64.57,23.49,11.93,35.43,1
1,20L,systematic,20.0,3.418188e+08,16.012180,660654.66,867392.82,929967.54,660654.66,206738.16,62574.72,269312.88,71.04,22.23,6.73,28.96,1
2,QC-8,systematic,69.3,1.275803e+08,5.976379,254606.34,285646.49,318072.07,254606.34,31040.15,32425.58,63465.73,80.05,9.76,10.19,19.95,10
3,QC-15,systematic,101.3,1.824364e+08,8.546061,364746.41,413499.04,461842.35,364746.41,48752.63,48343.31,97095.94,78.98,10.56,10.47,21.02,15
4,QC-30,systematic,212.6,3.773588e+08,17.677019,753703.13,850226.72,947535.12,753703.13,96523.59,97308.40,193831.99,79.54,10.19,10.27,20.46,31
5,QC-46,systematic,317.0,5.717800e+08,26.784498,1141892.75,1285443.16,1430139.18,1141892.75,143550.41,144696.02,288246.43,79.84,10.04,10.12,20.16,46
6,QC-61,systematic,419.9,7.543582e+08,35.337199,1507239.70,1699436.85,1892248.88,1507239.70,192197.15,192812.03,385009.18,79.65,10.16,10.19,20.35,61
